In [1]:
#!pip install tensorflow
#!pip install tensorflow_hub
import os
import time
from PIL import Image
from skimage import io
from skimage.transform import resize
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [2]:
def png(image):
    band = io.imread(image)
    band = np.squeeze(band[:,:,0])
    GSband = np.zeros((band.shape[0], band.shape[1], 3))
    GSband[:,:,0]= band
    GSband[:,:,1]= band
    GSband[:,:,2]= band
    GSband = GSband.astype('uint8')
    png = Image.fromarray(GSband)
    image = image[:-4]
    png.save(image + ".png")

#for filename in os.listdir('Fires/TIFF_Files/CampFire'):
#     png("Fires/TIFF_Files/CampFire/" + filename)

In [12]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

In [3]:
def preprocess_image(image_path):
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def downscale_image(image):
  image_size = []
  if len(image.shape) == 3:
    image_size = [image.shape[1], image.shape[0]]
  else:
    raise ValueError("Dimension mismatch. Can work only on single image.")
  image = tf.squeeze(tf.cast(tf.clip_by_value(image, 0, 255), tf.uint8))
  lr_image = np.asarray(Image.fromarray(image.numpy()).resize([image_size[0] // 2, image_size[1] // 2],Image.BICUBIC))
  lr_image = tf.expand_dims(lr_image, 0)
  lr_image = tf.cast(lr_image, tf.float32)
  return lr_image

In [14]:
table = []

hr_image = 0
fake_image = 0

In [15]:
def m_learning(fileID, band):
    
    hr_image = preprocess_image(fileID)
    hr_image = tf.squeeze(hr_image)
    lr_image = downscale_image(hr_image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)
    fake_image = resize(fake_image, (556, 540, 3))
    print('done')
    
    abs_error = abs(hr_image - fake_image)
    mean = np.mean(abs_error)
    rmse = np.sqrt(np.mean((hr_image - fake_image)**2))
    table.append({'Filename': fileID,'Band': band,'Mean': mean, 'RMSE': rmse})
    
    return pd.DataFrame(table)

In [ ]:
df07 = pd.concat(m_learning(str(file), '07') for file in Path('PNGs/').glob('GOES-17_ABI_RadC_C07*'))
df14 = pd.concat(m_learning(str(file), '14') for file in Path('PNGs/').glob('GOES-17_ABI_RadC_C14*'))

done
done
done
done
done


In [ ]:
#df_all = pd.concat([df14, df07])
#df_all.to_csv('data.csv')

In [ ]:
x = df_all.groupby(['Band']).std()

In [ ]:
x

In [4]:
df_all = pd.read_csv("data.csv")

In [7]:
df_all

,Unnamed: 0,Filename,Band,Mean,RMSE
0,0,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.332050,1.820826
1,1,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.237283,1.740089
2,2,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.315958,1.793417
3,3,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.402152,1.934412
4,4,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.250406,1.683789
...,...,...,...,...,...
1825,25,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.412189,1.967320
1826,26,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.281800,1.738358
1827,27,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.399028,1.953365
1828,28,Fires/Machine Learning/PNGs/GOES-17_ABI_RadC_C...,7,1.261871,1.785128


In [5]:
x = df_all.groupby(['Band']).std()

In [6]:
x

,Unnamed: 0,Mean,RMSE
Band,,,
7,8.500504,0.097117,0.139841
14,7.187953,0.177363,0.259998
